# Bluesky Subscriptions

See https://nsls-ii.github.io/bluesky for proper documentation.

Users walk in on an open IPython session that has already run a bunch of startup scripts. These scripts create an instance of a RunEngine, probably set some metadata, connect the RunEngine to metadatastore. They also define scan objects that pre-configured to dispatch the results from the scan to "Document consumers" that do thing like print to the screen and make plots. Finally, they define instances of Python objects that represent hardware. Currently, all of the objects come from the ophyd package, but they could also be from pyepics directly or some other package. They just have to have methods like `read()`.

In [0]:
%matplotlib notebook
from bluesky.examples import motor, det
from bluesky.run_engine import RunEngine
import bluesky.scans
from bluesky.callbacks import CallbackBase, LiveTable, LivePlot

In [0]:
RE = RunEngine({'owner': 'demo', 'group': 'demo', 'beamline_id': 'demo'})  # bare minimum required metadata

In [0]:
# Define a plan.
plan = bluesky.scans.AbsScan([det], motor, 1, 5, 5)  # This obj encapsulates the instruction for a kind of scan.

In [0]:
# Execute the plan.
RE(plan)

That was fine, but we got no feedback. Document were generated but they went nowhere. They are gone.

In [0]:
# One way to set up a subscription is to associate with a scan object.
plan.subs = LiveTable(['det'])

In [0]:
RE(plan)

In [0]:
# We can have as many subscriptions as we want.
plan.subs = [LiveTable(['det']), LivePlot('det')]
# Calling LivePlot creates the figure. When a run a scan with ascan, it will be filled with data.

In [0]:
RE(plan)

In [0]:
RE(plan)

In [0]:
def my_callback(name, doc):
    "a very simple custom callback"
    if name == 'start':
        print('My scan id is', doc['scan_id'])
    elif name == 'event':
        print('This is event number', doc['seq_num'])
    elif name == 'stop':
        print('We are done')

In [0]:
RE(plan, subs=my_callback)

In [0]:
class MyLiveCallback(CallbackBase):
    """
    The CallbackBase class dispatches each kind of document
    to an associated method.
    """
    
    def start(self, doc):
        print(doc)
    
    def event(self, doc):
        print(doc)
    
    def descriptor(self, doc):
        print(doc)
    
    def stop(self, doc):
        print(doc)

In [0]:
# To use a subscription for just a particular run -- not every time you run 'ascan' -- pass it as an argument like so.
RE(plan, subs=MyLiveCallback())